In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import numpy as np
from pytorch_grad_cam import GradCAMPlusPlus
from pytorch_grad_cam.utils.image import show_cam_on_image
import torch
import yaml
from torch import nn
import timm
from albumentations.pytorch.transforms import ToTensorV2
import albumentations as A

In [ ]:
# TODO: add GradCam ++
import plotly.express as px


def return_filpath(name, folder):
    """
    This function is used to return the file path of the image.
    :param name:
    :param folder:
    :return:
    """
    path = os.path.join(folder, f'{name}')
    return path


device = torch.device('cpu')
oof = pd.read_csv('/home/mithil/PycharmProjects/Cultivar_FGVC9/oof/tf_efficientnetv2_m_tta_oof.csv')
oof_probability = np.load('/home/mithil/PycharmProjects/Cultivar_FGVC9/oof/tf_efficientnetv2_m_tta_oof_probablity.npy')
oof['file_path'] = oof['image_id'].apply(
    lambda x: return_filpath(x, folder='/home/mithil/PycharmProjects/Cultivar_FGVC9/data/archive/train'))
images_path = oof['file_path'].values
ids = {}
truth_ids = []
for i, (pred, truth, pred_int) in enumerate(zip(oof['prediction'].values, oof['cultivar'].values, oof['cultivar_int'])):
    if pred != truth:
        ids.update({i: oof['file_path'].values[i]})
        truth_ids.append(truth)
print(len(ids))


wrong_preds_df = pd.DataFrame.from_dict(ids, orient='index')
wrong_preds_df['cultivar'] = truth_ids
wrong_ids = wrong_preds_df['cultivar'].value_counts().to_dict()
print(wrong_ids)
dict = {'x': list(wrong_ids.keys()), 'y': list(wrong_ids.values())}
fig = px.bar(dict, x='x', y='y', title='Wrong Predictions')
fig.show()

In [ ]:
import time

for key, values in ids.items():
    fig = plt.figure(figsize=(10, 10))
    ax = fig.subplots(1, 1)
    img = cv2.imread(values)

    ax.imshow(img)
    ax.set_title(
        f'Predicted: {oof["prediction"].values[key]},  Truth: {oof["cultivar"].values[key]}, Confidence : {np.max(oof_probability[key]):.3f}')

    plt.show()

    time.sleep(5)